In [1]:
import sys
import time

sys.path.append("..")  # make sure we can run this from the repo
from IPython.display import Markdown, clear_output, display
from ipywidgets import widgets, Layout

import cohere

## Client example

### Simple Generate Example

In [2]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.generate(prompt="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

[0][0.41s] StreamingText(id='0e3baf51-52ab-4ba9-8743-378cb75533e9', index=0, text='"')
[1][0.44s] StreamingText(id=None, index=0, text='\n')
[2][0.47s] StreamingText(id=None, index=0, text='\n')
[3][0.49s] StreamingText(id=None, index=0, text='I')
[4][0.55s] StreamingText(id=None, index=0, text=' smiled')
[5][0.58s] StreamingText(id=None, index=0, text=',')
[6][0.58s] StreamingText(id=None, index=0, text=' but')
[7][0.61s] StreamingText(id=None, index=0, text=' the')
[8][0.64s] StreamingText(id=None, index=0, text=' smile')
[9][0.66s] StreamingText(id=None, index=0, text=' only')
[10][0.69s] StreamingText(id=None, index=0, text=' lasted')
[11][0.72s] StreamingText(id=None, index=0, text=' for')
[12][0.75s] StreamingText(id=None, index=0, text=' a')
[13][0.77s] StreamingText(id=None, index=0, text=' second')
[14][0.80s] StreamingText(id=None, index=0, text=',')
[15][0.83s] StreamingText(id=None, index=0, text=' because')
[16][0.86s] StreamingText(id=None, index=0, text=' I')
[17][0.88s]

(['0e3baf51-52ab-4ba9-8743-378cb75533e9'],
 ['"\n\nI smiled, but the smile only lasted for a second, because I realized what I'])

### Widgets Chat Example

In [3]:
co = cohere.Client()
max_tokens = 250

text = ""
output_area = display("Type your message below and hit enter to chat!", display_id=True)
input_text = widgets.Text(value="Hey!", description="User:", disabled=False)

def append_text(s):
    global text
    text += s
    output_area.update(Markdown(text))

def stream_response(input_text):
    query = input_text.value
    if query.startswith("<generating"):
        return
    input_text.value = "<generating response>"
    append_text(f"\n\nUser: {query}\n\nChatbot: ")
    streaming_chat = co.chat(
        query=query,
        max_tokens=max_tokens,
        stream=True,
    )
    for token in streaming_chat:
        append_text(token.text)
    input_text.value = ""

display(input_text)
input_text.on_submit(stream_response)

'Type your message below and hit enter to chat!'

Text(value='Hey!', description='User:')

## AsyncClient example

### Widgets generate example

In [4]:
prompt = "안녕하"
num_generations = 5
max_tokens = 50
texts = [f"**Generation #{i+1}:** {prompt}" for i in range(num_generations)]
displays = [display(display_id=True) for t in texts]

async with cohere.AsyncClient() as aio_co:
    start_time = time.time()
    aio_streaming_gens = await aio_co.generate(
        prompt=prompt,
        max_tokens=max_tokens,
        model="command-xlarge-nightly",
        num_generations=num_generations,
        stream=True,
    )

    async for token in aio_streaming_gens:
        texts[token.index] += token.text
        displays[token.index].update(Markdown(texts[token.index]))

**Generation #1:** 안녕하세요.
I hope you're doing well.

8. 와, 다음에 공부할 걸 가지고 있어요.
Okay, I

**Generation #2:** 안녕하세요!
I am a native Korean who has been working in the Korean-English translation field for more than 5 years. My major is master's degree in linguistics. I am certified as a Korean language teacher and currently teaching Korean

**Generation #3:** 안녕하세요. 저는 걸프릴 수 있는 것이라 생각하고 있습니다. 그렇지만 교회

**Generation #4:** 안녕하세요~ 
My name is Dr. Brian Allen, and I am a professor at the University of Wisconsin-Madison. I am contacting you today because I am interested in learning more about your research on the cognitive and affective processing

**Generation #5:** 안녕하세요!
I am a native Korean speaker, and I have been teaching Korean to students of all ages and levels since 2008. I received my MA degree in Korean Language and Linguistics in 2009. I currently teach at the Korean

### Simple Chat Example

In [5]:
query = "Do You Want to Build a Snowman?"
async with cohere.AsyncClient() as aio_co:
    streaming_chat = await aio_co.chat(
        query=query,
        max_tokens=50,
        stream=True,
    )
    async for token in streaming_chat:
        print(token)
    print(streaming_chat.texts)

StreamingText(index=0, text='Yes')
StreamingText(index=0, text=',')
StreamingText(index=0, text=' I')
StreamingText(index=0, text=' would')
StreamingText(index=0, text=' like')
StreamingText(index=0, text=' to')
StreamingText(index=0, text=' build')
StreamingText(index=0, text=' a')
StreamingText(index=0, text=' snow')
StreamingText(index=0, text='man')
StreamingText(index=0, text='.')
['Yes, I would like to build a snowman.']
